# Use GPU

In [1]:
import os

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

# Loading Data

In [2]:
import numpy as np

data = np.load("FinalData/channel_last_data.npz")
normalData = data["input"]
shuffledData = data["input"]
np.random.shuffle(shuffledData)

trainInput,trainOutput = shuffledData[0:15000],shuffledData[0:15000]
testInput,testOutput = shuffledData[15000:],shuffledData[15000:]

# Helper Functions

In [3]:
import numpy as np
import Sokoban
import random
import tensorflow as tf
import keras.backend as K
from keras.losses import categorical_crossentropy

def normal_loss(y_true, y_pred):
    return K.mean(categorical_crossentropy(y_true, y_pred), axis=[1,2])

def numpyToString(array):
    gameCharacters="# @$.+*"
    char_to_int = dict((c, i) for i, c in enumerate(gameCharacters))
    int_to_char = dict((i, c) for i, c in enumerate(gameCharacters))
    intArray = np.argmax(array, axis=2)
    output=""
    for (i,j), index in np.ndenumerate(intArray):
        if i > 0 and j == 0:
            output += "\n"
        output += int_to_char[index]
    return output

def convertStringTo2DArray(string):
    return np.array([list(l) for l in string.split("\n")])

def getOneHotEncodingMap(data):
    gameCharacters="# @$.+*"
    char_to_int = dict((c, i) for i, c in enumerate(gameCharacters))
    int_to_char = dict((i, c) for i, c in enumerate(gameCharacters))

    encodingData = np.zeros((data.shape[0], data.shape[1], len(gameCharacters)))
    for (i,j),c in np.ndenumerate(data):
        index=char_to_int[c]
        encodingData[i][j][index]=1
    return encodingData

def getHammingDistance(data1, data2):
    data1=np.argmax(data1, axis=3)
    data2=np.argmax(data2, axis=3)
    result=np.absolute(data1-data2)
    result=np.clip(result,0,1)
    return np.sum(result)/data1.shape[0]

def combineTwoStrings(str1, str2):
    if len(str1) == 0:
        return str2
    if len(str2) == 0:
        return str1
    lines1=str1.split("\n")
    lines2=str2.split("\n")
    linesOut=[]
    for i in range(10):
        linesOut.append(lines1[i] + "  " + lines2[i])
    return "\n".join(linesOut)

def getNoisedState(numpyLevel, randomSize=20):
    stringLvl=numpyToString(numpyLevel)
    state=Sokoban.State()
    state.stringInitialize(stringLvl.split("\n"))
    for i in range(randomSize):
        if random.random() < 0.5:
            state.update(0, 2*random.randint(0,1) - 1)
        else:
            state.update(2*random.randint(0,1) - 1, 0)
    return getOneHotEncodingMap(convertStringTo2DArray(str(state)))

/home/jupyter/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jupyter/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:520: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jupyter/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:521: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jupyter/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework

# Testing Different Encoding Sizes

In [4]:
import os
import datetime
import keras.layers as layers
from keras.models import Model
from keras.callbacks import ModelCheckpoint, TensorBoard, LambdaCallback
import keras.backend as K
import tensorflow as tf

encoding_sizes = [32, 16, 8, 4]

for encoding in encoding_sizes:
    input_level=layers.Input(shape=(10,10,7))
    encoded_level=layers.Input(shape=(encoding,))

    x = layers.ZeroPadding2D(padding=(1,1))(input_level)
    x = layers.Conv2D(128, (3,3), activation="relu", padding="same")(x)
    x = layers.MaxPooling2D((2,2), padding="same")(x)
    x = layers.Conv2D(64, (3,3), activation="relu", padding="same")(x)
    x = layers.MaxPooling2D((2,2), padding="same")(x)
    x = layers.Conv2D(32, (3,3), activation="relu", padding="same")(x)
    x = layers.MaxPooling2D((3,3), padding="same")(x)
    x = layers.Dense(encoding, activation="relu")(x)
    encoded = layers.Flatten()(x)

    x = layers.Reshape((1,1,encoding))(encoded)
    x = layers.Dense(32, activation="relu")(x)
    x = layers.UpSampling2D((3,3))(x)
    x = layers.Conv2D(64, (3,3), activation="relu", padding="same")(x)
    x = layers.UpSampling2D((2,2))(x)
    x = layers.Conv2D(128, (3,3), activation="relu", padding="same")(x)
    x = layers.UpSampling2D((2,2))(x)
    x = layers.Conv2D(7, (3,3), padding="valid")(x)
    decoded = layers.Softmax(axis=3)(x)

    autoencoder = Model(input_level, decoded)
    encoder = Model(input_level, encoded)
    decoded = autoencoder.layers[-9](encoded_level)
    decoded = autoencoder.layers[-8](decoded)
    decoded = autoencoder.layers[-7](decoded)
    decoded = autoencoder.layers[-6](decoded)
    decoded = autoencoder.layers[-5](decoded)
    decoded = autoencoder.layers[-4](decoded)
    decoded = autoencoder.layers[-3](decoded)
    decoded = autoencoder.layers[-2](decoded)
    decoded = autoencoder.layers[-1](decoded)
    decoder = Model(encoded_level,decoded)

    autoencoder.compile(optimizer='adam', loss=normal_loss)
    autoencoder.summary()
    
#     folder_path = "Weights/Encoding_" + str(encoding) + "/"
#     os.mkdir(folder_path)
    
#     log_dir="logs/fit/encoding_" + str(encoding) + "_images_" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
#     writer = tf.summary.FileWriter(log_dir + "_encoding_mag")
#     tensorboard_callback = TensorBoard(log_dir=log_dir)
#     file_writer = tf.summary.create_file_writer(log_dir + "/metrics")
#     file_writer.set_as_default()
#     def recordSummary(epoch, logs):
#         encoding_mag = K.mean(K.sqrt(K.sum(K.square(encoded), axis=1)), axis=0)
#         tf.summary.scalar('encoding_mag', data=encoding_mag, step=epoch)
#     summary_callback = LambdaCallback(on_train_batch_end=recordSummary)
    
    autoencoder.fit(trainInput, trainOutput
                ,epochs=2000
                ,batch_size=128
                ,shuffle=True
                ,validation_data=(testInput, testOutput)
#                 ,verbose=1
#                 ,callbacks=[ModelCheckpoint(folder_path + "model_{epoch:02d}_{loss:.2f}_{val_loss:.2f}.hdf5", 
#                                             save_weights_only=False, save_best_only=False, save_freq=20), 
#                             tensorboard_callback]
               )

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 10, 10, 7)         0         
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 12, 12, 7)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 12, 12, 128)       8192      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 128)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 6, 6, 64)          73792     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 3, 3, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 32)          18464     
__________

15000/15000 [==============================] - 1s 71us/step - loss: 0.2007 - val_loss: 0.2190
Epoch 51/2000
15000/15000 [==============================] - 1s 69us/step - loss: 0.1991 - val_loss: 0.2168
Epoch 52/2000
15000/15000 [==============================] - 1s 68us/step - loss: 0.1979 - val_loss: 0.2169
Epoch 53/2000
15000/15000 [==============================] - 1s 68us/step - loss: 0.1954 - val_loss: 0.2140
Epoch 54/2000
15000/15000 [==============================] - 1s 69us/step - loss: 0.1948 - val_loss: 0.2187
Epoch 55/2000
15000/15000 [==============================] - 1s 70us/step - loss: 0.1931 - val_loss: 0.2141
Epoch 56/2000
15000/15000 [==============================] - 1s 70us/step - loss: 0.1914 - val_loss: 0.2136
Epoch 57/2000
15000/15000 [==============================] - 1s 69us/step - loss: 0.1924 - val_loss: 0.2089
Epoch 58/2000
15000/15000 [==============================] - 1s 68us/step - loss: 0.1911 - val_loss: 0.2172
Epoch 59/2000
15000/15000 [===============

15000/15000 [==============================] - 1s 69us/step - loss: 0.1421 - val_loss: 0.1872
Epoch 126/2000
15000/15000 [==============================] - 1s 69us/step - loss: 0.1486 - val_loss: 0.1856
Epoch 127/2000
15000/15000 [==============================] - 1s 71us/step - loss: 0.1418 - val_loss: 0.1866
Epoch 128/2000
15000/15000 [==============================] - 1s 70us/step - loss: 0.1414 - val_loss: 0.1830
Epoch 129/2000
15000/15000 [==============================] - 1s 68us/step - loss: 0.1392 - val_loss: 0.1841
Epoch 130/2000
15000/15000 [==============================] - 1s 69us/step - loss: 0.1559 - val_loss: 0.2275
Epoch 131/2000
15000/15000 [==============================] - 1s 71us/step - loss: 0.1480 - val_loss: 0.1814
Epoch 132/2000
15000/15000 [==============================] - 1s 69us/step - loss: 0.1374 - val_loss: 0.1812
Epoch 133/2000
15000/15000 [==============================] - 1s 70us/step - loss: 0.1372 - val_loss: 0.1804
Epoch 134/2000
15000/15000 [======

15000/15000 [==============================] - 1s 67us/step - loss: 0.1196 - val_loss: 0.1886
Epoch 201/2000
15000/15000 [==============================] - 1s 70us/step - loss: 0.1211 - val_loss: 0.1914
Epoch 202/2000
15000/15000 [==============================] - 1s 67us/step - loss: 0.1326 - val_loss: 0.1882
Epoch 203/2000
15000/15000 [==============================] - 1s 69us/step - loss: 0.1194 - val_loss: 0.1867
Epoch 204/2000
15000/15000 [==============================] - 1s 68us/step - loss: 0.1171 - val_loss: 0.1880
Epoch 205/2000
15000/15000 [==============================] - 1s 69us/step - loss: 0.1174 - val_loss: 0.1897
Epoch 206/2000
15000/15000 [==============================] - 1s 70us/step - loss: 0.1185 - val_loss: 0.1889
Epoch 207/2000
15000/15000 [==============================] - 1s 71us/step - loss: 0.1193 - val_loss: 0.1980
Epoch 208/2000
15000/15000 [==============================] - 1s 69us/step - loss: 0.1199 - val_loss: 0.1847
Epoch 209/2000
15000/15000 [======

15000/15000 [==============================] - 1s 69us/step - loss: 0.1061 - val_loss: 0.2012
Epoch 276/2000
15000/15000 [==============================] - 1s 72us/step - loss: 0.1105 - val_loss: 0.1934
Epoch 277/2000
15000/15000 [==============================] - 1s 67us/step - loss: 0.1060 - val_loss: 0.1989
Epoch 278/2000
15000/15000 [==============================] - 1s 70us/step - loss: 0.1062 - val_loss: 0.1971
Epoch 279/2000
15000/15000 [==============================] - 1s 70us/step - loss: 0.1097 - val_loss: 0.2051
Epoch 280/2000
15000/15000 [==============================] - 1s 68us/step - loss: 0.1134 - val_loss: 0.1955
Epoch 281/2000
15000/15000 [==============================] - 1s 69us/step - loss: 0.1063 - val_loss: 0.1971
Epoch 282/2000
15000/15000 [==============================] - 1s 69us/step - loss: 0.1086 - val_loss: 0.1980
Epoch 283/2000
15000/15000 [==============================] - 1s 69us/step - loss: 0.1059 - val_loss: 0.1902
Epoch 284/2000
15000/15000 [======

...

15000/15000 [==============================] - 1s 68us/step - loss: 0.1777 - val_loss: 0.2587
Epoch 338/2000
15000/15000 [==============================] - 1s 68us/step - loss: 0.1780 - val_loss: 0.2591
Epoch 339/2000
15000/15000 [==============================] - 1s 69us/step - loss: 0.1757 - val_loss: 0.2767
Epoch 340/2000
15000/15000 [==============================] - 1s 69us/step - loss: 0.1769 - val_loss: 0.2599
Epoch 341/2000
15000/15000 [==============================] - 1s 69us/step - loss: 0.1796 - val_loss: 0.2591
Epoch 342/2000
15000/15000 [==============================] - 1s 69us/step - loss: 0.1756 - val_loss: 0.2639
Epoch 343/2000
15000/15000 [==============================] - 1s 68us/step - loss: 0.1784 - val_loss: 0.2658
Epoch 344/2000
15000/15000 [==============================] - 1s 69us/step - loss: 0.1801 - val_loss: 0.2661
Epoch 345/2000
15000/15000 [==============================] - 1s 68us/step - loss: 0.1773 - val_loss: 0.2645
Epoch 346/2000
15000/15000 [======

15000/15000 [==============================] - 1s 71us/step - loss: 0.1737 - val_loss: 0.2711
Epoch 413/2000
15000/15000 [==============================] - 1s 68us/step - loss: 0.1710 - val_loss: 0.2681
Epoch 414/2000
15000/15000 [==============================] - 1s 69us/step - loss: 0.1714 - val_loss: 0.2646
Epoch 415/2000
15000/15000 [==============================] - 1s 69us/step - loss: 0.1781 - val_loss: 0.2818
Epoch 416/2000
15000/15000 [==============================] - 1s 70us/step - loss: 0.1771 - val_loss: 0.2680
Epoch 417/2000
15000/15000 [==============================] - 1s 69us/step - loss: 0.1727 - val_loss: 0.2663
Epoch 418/2000
15000/15000 [==============================] - 1s 69us/step - loss: 0.1701 - val_loss: 0.2684
Epoch 419/2000
15000/15000 [==============================] - 1s 69us/step - loss: 0.1717 - val_loss: 0.2812
Epoch 420/2000
15000/15000 [==============================] - 1s 70us/step - loss: 0.1797 - val_loss: 0.2683
Epoch 421/2000
15000/15000 [======

15000/15000 [==============================] - 1s 69us/step - loss: 0.1666 - val_loss: 0.2726
Epoch 488/2000
15000/15000 [==============================] - 1s 68us/step - loss: 0.1659 - val_loss: 0.2748
Epoch 489/2000
15000/15000 [==============================] - 1s 70us/step - loss: 0.1675 - val_loss: 0.2722
Epoch 490/2000
15000/15000 [==============================] - 1s 68us/step - loss: 0.1699 - val_loss: 0.2796
Epoch 491/2000
15000/15000 [==============================] - 1s 68us/step - loss: 0.1683 - val_loss: 0.2747
Epoch 492/2000
15000/15000 [==============================] - 1s 68us/step - loss: 0.1676 - val_loss: 0.2751
Epoch 493/2000
15000/15000 [==============================] - 1s 68us/step - loss: 0.1668 - val_loss: 0.2776
Epoch 494/2000
15000/15000 [==============================] - 1s 70us/step - loss: 0.1717 - val_loss: 0.2727
Epoch 495/2000
15000/15000 [==============================] - 1s 71us/step - loss: 0.1708 - val_loss: 0.2873
Epoch 496/2000
15000/15000 [======

15000/15000 [==============================] - 1s 69us/step - loss: 0.1633 - val_loss: 0.2780
Epoch 563/2000
15000/15000 [==============================] - 1s 69us/step - loss: 0.1648 - val_loss: 0.2875
Epoch 564/2000
15000/15000 [==============================] - 1s 69us/step - loss: 0.1679 - val_loss: 0.2795
Epoch 565/2000
15000/15000 [==============================] - 1s 69us/step - loss: 0.1662 - val_loss: 0.2901
Epoch 566/2000
15000/15000 [==============================] - 1s 71us/step - loss: 0.1688 - val_loss: 0.2793
Epoch 567/2000
15000/15000 [==============================] - 1s 68us/step - loss: 0.1641 - val_loss: 0.2811
Epoch 568/2000
15000/15000 [==============================] - 1s 67us/step - loss: 0.1656 - val_loss: 0.2810
Epoch 569/2000
15000/15000 [==============================] - 1s 68us/step - loss: 0.1629 - val_loss: 0.2862
Epoch 570/2000
15000/15000 [==============================] - 1s 69us/step - loss: 0.1632 - val_loss: 0.2782
Epoch 571/2000
15000/15000 [======

KeyboardInterrupt: 

# Sampling Function

In [ ]:
import os
import Sokoban
import keras.layers as layers
from keras.models import Model
from keras.callbacks import ModelCheckpoint

encoding_sizes = [4, 8, 16, 32, 64]

for encoding in encoding_sizes:
    input_level=layers.Input(shape=(10,10,7))
    encoded_level=layers.Input(shape=(encoding,))

    x = layers.ZeroPadding2D(padding=(1,1))(input_level)
    x = layers.Conv2D(128, (3,3), activation="relu", padding="same")(x)
    x = layers.MaxPooling2D((4,4), padding="same")(x)
    x = layers.Conv2D(64, (3,3), activation="relu", padding="same")(x)
    x = layers.MaxPooling2D((3,3), padding="same")(x)
    x = layers.Dense(encoding, activation="sigmoid", name='encoded')(x)
    encoded = layers.Flatten()(x)

    x = layers.Reshape((1,1,encoding))(encoded)
    x = layers.UpSampling2D((3,3))(x)
    x = layers.Conv2D(64, (3,3), activation="relu", padding="same")(x)
    x = layers.UpSampling2D((2,2))(x)
    x = layers.Conv2D(128, (3,3), activation="relu", padding="same")(x)
    x = layers.UpSampling2D((2,2))(x)
    x = layers.Conv2D(7, (3,3), padding="valid")(x)
    decoded = layers.Softmax(axis=3)(x)

    autoencoder = Model(input_level, decoded)
    encoder = Model(input_level, encoded)
    decoded = autoencoder.layers[-8](encoded_level)
    decoded = autoencoder.layers[-7](decoded)
    decoded = autoencoder.layers[-6](decoded)
    decoded = autoencoder.layers[-5](decoded)
    decoded = autoencoder.layers[-4](decoded)
    decoded = autoencoder.layers[-3](decoded)
    decoded = autoencoder.layers[-2](decoded)
    decoded = autoencoder.layers[-1](decoded)
    decoder = Model(encoded_level,decoded)

    autoencoder.compile(optimizer='adam', loss=normal_loss)
    autoencoder.summary()
    
    files = os.listdir("Weights/Encoding_" + str(encoding) + "/")
    for f in files:
        if "2000" in f:
            autoencoder.autoencoder.load_weights("Weights/Encoding_" + str(encoding) + "/" + f)
            for i in range()

In [5]:
import time
import Sokoban
import imp
imp.reload(Sokoban)

%matplotlib inline
import matplotlib.pyplot as plt


stringLvl=numpyToString(normalData[0])
print(stringLvl)
state=Sokoban.State()
state.stringInitialize(stringLvl.split("\n"))
aStarAgent = Sokoban.AStarAgent()
enaStarAgent = Sokoban.EnhancedAStarAgent()
bfsAgent = Sokoban.BFSAgent()
dfsAgent = Sokoban.DFSAgent()

image = state.getImage()
import io
output = io.BytesIO()
image.save(output, format='PNG')
image_string = output.getvalue()
output.close()

start = time.process_time()
sol,solState,iters = bfsAgent.getSolution(state, 500000)
print("BFS: " + str(solState.checkWin()) + " " + str(len(sol)) + " " + str(iters))
print(time.process_time() - start)

start = time.process_time()
sol,solState,iters = enaStarAgent.getSolution(state, 1, 100000)
print("HeapQ AStar 1.0: " + str(solState.checkWin()) + " " + str(len(sol)) + " " + str(iters))
print(time.process_time() - start)

# start = time.process_time()
# sol,solState,iters = enaStarAgent.getSolution(state, 0.5, 100000)
# print("HeapQ AStar 0.5: " + str(solState.checkWin()) + " " + str(len(sol)) + " " + str(iters))
# print(time.process_time() - start)

# start = time.process_time()
# sol,solState,iters = enaStarAgent.getSolution(state, 0.25, 100000)
# print("HeapQ AStar 0.25: " + str(solState.checkWin()) + " " + str(len(sol)) + " " + str(iters))
# print(time.process_time() - start)

start = time.process_time()
sol,solState,iters = enaStarAgent.getSolution(state, 0, 100000)
print("HeapQ AStar 0.0: " + str(solState.checkWin()) + " " + str(len(sol)) + " " + str(iters))
print(time.process_time() - start)

##########
# .#######
#  #######
#*@  #####
#  $ #####
#  #######
##########
##########
##########
##########
b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x00\xa0\x00\x00\x00\xa0\x08\x06\x00\x00\x00\x8b\xcfg-\x00\x00\x04\xedIDATx\x9c\xed\xdd1N\x1bM\x18\xc6\xf1\x97\x98[`\xd9\x9dQh\xe8s\x05\x8b\x88;\xb8\x88\xa8,\xb9\xe0\x10\x14\x91\xdc\x91"w\xb0\x14q\x05zR\x04\x91n\x11\x94\x1c\xc1"\x05Z\xc7,c\xbc\xcc\x8c\xfc\xcc\xec\xfc\x7fR\xe4/\x11o\x16)\x8f\x1e\xcf\xda\xaf\xf9\xf6\x9e\x9f\xce\x9e\xcd\xd3\xed\xc9\x8d}\xfeu\xec;\xce<\xf3\xb6_\xff\x87\x8fyU\xd9\xa5\x1d3\xcf\xbc\xf7\xfc\'\xafI \x12\x02\x08)\x02\x08)\x02\x08)\x02\x08)\x02\x08)\x02\x08)\x02\x08)\x02\x08)\x02\x08)\x02\x08)\x02\x08)\x02\x08\xa9\xfd\xdb\x93\x1b\x9bW\x95\xf7_\xc0<\xf3!\xf3{\xa1\xfb\x80\xbd\xd3\x07\xef\x8b\xc70\x9a\x8c\xbdgS\xd8\x87+}>|\x1fp\xf2\xd5\xfe\xfe\xbc\xf2\xfe&F\x93q\xd0\xbc\x19\xfb\x8c9\xcf\x07\x9f\x01C\xc3S\xcf\x1f\x9e\xdf{="o\xc1\x01\x0cy\n\\\x9f\xbf\xbb\x18x="oI4 \xedW\xae$\x1a\x90\xf6+W\x12\rh\xc6\x19\xb0TI4\xa0\x

In [ ]:
print(tensorboard.version.VERSION)